In [1]:
import os
import sys
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'
os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 2 pyspark-shell'

spark_home = os.environ.get('SPARK_HOME', None)
if not spark_home:
    raise ValueError('SPARK_HOME environment variable is not set')

sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))
exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.4.7
      /_/

Using Python version 3.6.5 (default, Apr 29 2018 16:14:56)
SparkSession available as 'spark'.


In [5]:
from pyspark import SparkConf, HiveContext
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark import Row

conf = SparkConf().setMaster("yarn").setAppName("lab01")
sc = SparkContext.getOrCreate(conf)
spark = HiveContext(sc)

print(sc.getConf().get("spark.app.id"))

application_1665753715559_1221


In [6]:
import json
from collections import Counter, OrderedDict

In [7]:
film_id = 302

In [8]:
lines = sc.textFile("/labs/laba01/ml-100k/u.data")

In [9]:
films = lines.map(lambda x: x.split("\t", 3)[1:3]).map(lambda x: tuple(map(int, x)))

In [10]:
hist_film = films.filter(lambda x: x[0] == film_id).map(lambda x: (x[1], 1)).reduceByKey(lambda x, y: x + y).sortBy(lambda x: x[1])

In [11]:
hist_film_dict = OrderedDict(sorted(hist_film.collect()))
hist_film_dict

OrderedDict([(1, 2), (2, 10), (3, 46), (4, 119), (5, 120)])

In [12]:
hist_all = films.map(lambda x: (x[1], 1)).reduceByKey(lambda x, y: x + y).sortBy(lambda x: x[1])

In [13]:
hist_all_dict = OrderedDict(sorted(hist_all.collect()))
hist_all_dict

OrderedDict([(1, 6110), (2, 11370), (3, 27145), (4, 34174), (5, 21201)])

In [14]:
hist_json = dict(hist_film = list(hist_film_dict.values()), hist_all = list(hist_all_dict.values()))
hist_json

{'hist_film': [2, 10, 46, 119, 120],
 'hist_all': [6110, 11370, 27145, 34174, 21201]}

In [15]:
with open(os.path.join(os.environ["HOME"], "lab01.json"), "w") as f:
    json.dump(hist_json, f)

In [16]:
sc.stop()